# Feature Creation
### Author: Terence Kaplan

In [1]:
# Load necessary packages
import pandas as pd
import numpy as np

In [2]:
# Read the CSV file into a data frame
df = pd.read_csv('./HI-Small_Trans.csv')

In [3]:
# Convert variable "Timestamp" to date and time and arrange in ascending order
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')
df = df.sort_values('Timestamp')

In [4]:
# Rename variables "Account" to "From Account" and "Account.1" to "To Account"
df = df.rename(columns={"Account": "From Account", "Account.1": "To Account"})

### Descriptive Statistics

In [5]:
# Inspect the data set
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5078345 entries, 316720 to 4962234
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Timestamp           datetime64[ns]
 1   From Bank           int64         
 2   From Account        object        
 3   To Bank             int64         
 4   To Account          object        
 5   Amount Received     float64       
 6   Receiving Currency  object        
 7   Amount Paid         float64       
 8   Payment Currency    object        
 9   Payment Format      object        
 10  Is Laundering       int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 464.9+ MB


In [6]:
# Get length of timespan
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2022-09-01 00:00:00'), Timestamp('2022-09-18 16:18:00'))

In [7]:
# Number and ratio of non-fraud and fraud cases
occ = df['Is Laundering'].value_counts()
occ, occ / len(df.index)

(0    5073168
 1       5177
 Name: Is Laundering, dtype: int64,
 0    0.998981
 1    0.001019
 Name: Is Laundering, dtype: float64)

In [8]:
# Count number of unique accounts
all_accounts = pd.concat([df['From Account'], df['To Account']])
all_accounts.nunique()

515080

In [9]:
# Count number of unique banks
all_banks = pd.concat([df['From Bank'], df['To Bank']])
all_banks.nunique()

30470

In [10]:
# Calculate number of transactions for each currency

# For all observations
currency_counts = df['Payment Currency'].value_counts().sort_values(ascending=False)
currency_percentage = ((currency_counts / len(df)) * 100).round(1)

# For observations grouped by class

# Calculate number of transactions
currency_grouped_counts = df.groupby(['Is Laundering', 'Payment Currency']).size()

# Calculate total number of transactions for each class
currency_total_counts = df.groupby('Is Laundering').size()

# Calculate percentages for each class
currency_grouped_percentage = (currency_grouped_counts / currency_total_counts * 100).round(1)

# Merge all into one data frame
currency_counts_all = pd.DataFrame({
    'Currency': currency_counts.index,
    'Counts total': currency_counts.values,
    'Percentage total': currency_percentage.values,
    'Counts legitimate': currency_grouped_counts[0].reindex(currency_counts.index, fill_value=0).values,
    'Percentage legitimate': currency_grouped_percentage[0].reindex(currency_counts.index, fill_value=0).values,
    'Counts fraudulent': currency_grouped_counts[1].reindex(currency_counts.index, fill_value=0).values,
    'Percentage fraudulent': currency_grouped_percentage[1].reindex(currency_counts.index, fill_value=0).values
}).set_index('Currency')

currency_counts_all

,Counts total,Percentage total,Counts legitimate,Percentage legitimate,Counts fraudulent,Percentage fraudulent
Currency,,,,,,
US Dollar,1895172,37.3,1893260,37.3,1912,36.9
Euro,1168297,23.0,1166925,23.0,1372,26.5
Swiss Franc,234860,4.6,234667,4.6,193,3.7
Yuan,213752,4.2,213568,4.2,184,3.6
Shekel,192184,3.8,192089,3.8,95,1.8
Rupee,190202,3.7,190035,3.7,167,3.2
UK Pound,180738,3.6,180606,3.6,132,2.5
Yen,155209,3.1,155054,3.1,155,3.0
Ruble,155178,3.1,155045,3.1,133,2.6


In [11]:
# Calculate number of transactions for each payment format

# For all observations
format_counts = df['Payment Format'].value_counts().sort_values(ascending=False)
format_percentage = ((format_counts / len(df)) * 100).round(1)

# For observations grouped by class

# Calculate number of transactions
format_grouped_counts = df.groupby(['Is Laundering', 'Payment Format']).size()

# Calculate total number of transactions for each class
format_total_counts = df.groupby('Is Laundering').size()

# Calculate percentages for each class
format_grouped_percentage = (format_grouped_counts / format_total_counts * 100).round(1)

# Merge all into one data frame
format_counts_all = pd.DataFrame({
    'Format': format_counts.index,
    'Counts total': format_counts.values,
    'Percentage total': format_percentage.values,
    'Counts legitimate': format_grouped_counts[0].reindex(format_counts.index, fill_value=0).values,
    'Percentage legitimate': format_grouped_percentage[0].reindex(format_counts.index, fill_value=0).values,
    'Counts fraudulent': format_grouped_counts[1].reindex(format_counts.index, fill_value=0).values,
    'Percentage fraudulent': format_grouped_percentage[1].reindex(format_counts.index, fill_value=0).values
}).set_index('Format')

format_counts_all

,Counts total,Percentage total,Counts legitimate,Percentage legitimate,Counts fraudulent,Percentage fraudulent
Format,,,,,,
Cheque,1864331,36.7,1864007,36.7,324,6.3
Credit Card,1323324,26.1,1323118,26.1,206,4.0
ACH,600797,11.8,596314,11.8,4483,86.6
Cash,490891,9.7,490783,9.7,108,2.1
Reinvestment,481056,9.5,481056,9.5,0,0.0
Wire,171855,3.4,171855,3.4,0,0.0
Bitcoin,146091,2.9,146035,2.9,56,1.1


### V1: Average of the Digits Sums & Average Number of Zeros

In [12]:
# Create function to calculate the sum of digits of a transaction amount
def digit_sum(amount):
    return sum(int(digit) for digit in str(amount) if digit.isdigit())

# Create function to count the number of zeros of a transaction amount
def count_zeros(amount):
    formatted_amount = "{:.6f}".format(amount)
    return formatted_amount.count('0')

# Compute the sum of digits and the number of zeros for each transaction
df['Digit Sum Amount Received'] = df['Amount Received'].apply(digit_sum)
df['Digit Sum Amount Paid'] = df['Amount Paid'].apply(digit_sum)
df['Count Zeros Amount Received'] = df['Amount Received'].apply(count_zeros)
df['Count Zeros Amount Paid'] = df['Amount Paid'].apply(count_zeros)

# Calculate the average digit sum and average number of zeros for paid transactions by each account
paid_digit_stats = df.groupby('From Account').agg({
    'Digit Sum Amount Paid': np.mean,
    'Count Zeros Amount Paid': np.mean
}).reset_index()
paid_digit_stats.rename(columns={
    'Digit Sum Amount Paid': 'Average Digit Sum Paid',
    'Count Zeros Amount Paid': 'Average Count Zeros Paid'
}, inplace=True)

# Calculate the average digit sum and average number of zeros for received transactions by each account
received_digit_stats = df.groupby('To Account').agg({
    'Digit Sum Amount Received': np.mean,
    'Count Zeros Amount Received': np.mean
}).reset_index()
received_digit_stats.rename(columns={
    'Digit Sum Amount Received': 'Average Digit Sum Received',
    'Count Zeros Amount Received': 'Average Count Zeros Received'
}, inplace=True)

# Merge both data sets into a single data frame
digit_stats = pd.merge(paid_digit_stats, received_digit_stats, how='outer', left_on='From Account', right_on='To Account')

# Combine the two account IDs in a new variable "Account ID"
digit_stats['Account ID'] = digit_stats['From Account'].combine_first(digit_stats['To Account'])

# Drop redundant columns
digit_stats = digit_stats.drop(columns=['From Account', 'To Account'])
digit_stats.head()

,Average Digit Sum Paid,Average Count Zeros Paid,Average Digit Sum Received,Average Count Zeros Received,Account ID
0,23.920141,4.480584,20.681734,4.426199,100428660
1,23.354880,4.484886,19.860643,4.382848,1004286A8
2,27.371377,4.560092,24.629630,4.351852,1004286F0
3,32.610425,4.693297,30.081633,4.581633,100428738
4,32.097949,4.635832,29.290598,4.632479,100428780


In [13]:
# Merge with variable "From Account"
df_v1 = pd.merge(df, digit_stats, how='left', left_on='From Account', right_on='Account ID')
df_v1.rename(columns={
    'Average Digit Sum Paid': 'From Account Average Digit Sum Paid',
    'Average Count Zeros Paid': 'From Account Average Count Zeros Paid',
    'Average Digit Sum Received': 'From Account Average Digit Sum Received',
    'Average Count Zeros Received': 'From Account Average Count Zeros Received'
}, inplace=True)
df_v1.drop(columns=['Account ID'], inplace=True)

# Merge with variable "To Account"
df_v1 = pd.merge(df_v1, digit_stats, how='left', left_on='To Account', right_on='Account ID')
df_v1.rename(columns={
    'Average Digit Sum Paid': 'To Account Average Digit Sum Paid',
    'Average Count Zeros Paid': 'To Account Average Count Zeros Paid',
    'Average Digit Sum Received': 'To Account Average Digit Sum Received',
    'Average Count Zeros Received': 'To Account Average Count Zeros Received'
}, inplace=True)
df_v1.drop(columns=['Account ID'], inplace=True)
df_v1.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,47.64,Saudi Riyal,47.64,Saudi Riyal,Reinvestment,...,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,3917.42,Canadian Dollar,3917.42,Canadian Dollar,Reinvestment,...,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,97.49,Canadian Dollar,97.49,Canadian Dollar,Credit Card,...,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537
3,2022-09-01,32317,800D4E490,12004,800D4E750,13939.05,Euro,13939.05,Euro,Wire,...,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.500000
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.37,Euro,10.37,Euro,Reinvestment,...,5,5,19.333333,4.766667,19.250000,4.500000,19.333333,4.766667,19.250000,4.500000


In [14]:
# CHECK RESULTS - Average of the digit sums and average number of zeros by a selected account

# Check whether digit sums and number are correct calculated for a specific account
df_v1_test = df[df['From Account'] == '100428780']
df_v1_test.describe() # to find average

df_v1_test2 = df[df['To Account'] == '100428780']
df_v1_test2.describe() # to find average

# Check with the corresponding variables
df_v1_test2 = df_v1[df_v1['To Account'] == '100428780']
df_v1_test2.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received
114023,2022-09-01 00:10:00,313638,804F4AED0,70,100428780,0.11,Rupee,0.11,Rupee,Cash,...,5,5,16.666667,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479
124189,2022-09-01 00:11:00,312747,8054CB880,70,100428780,0.10,Rupee,0.10,Rupee,Cash,...,6,6,19.000000,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479
139045,2022-09-01 00:12:00,312246,805410180,70,100428780,0.03,Rupee,0.03,Rupee,Cash,...,6,6,23.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479
465982,2022-09-01 04:38:00,312830,80536CA40,70,100428780,0.08,Rupee,0.08,Rupee,Cash,...,6,6,22.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479
724918,2022-09-01 12:22:00,313167,804CDF110,70,100428780,0.47,Rupee,0.47,Rupee,Cash,...,5,5,14.333333,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479


### V2: Average Ratio of Zeros compared to Total Number of Digits

In [15]:
# Create function to calculate the ratio of zero digits to the total number of a transaction amount
def zero_digit_ratio(amount):
    amount_str = "{:.6f}".format(amount)
    return amount_str.count('0') / (len(amount_str) - int("." in amount_str))

# Compute the ratio of zero digits for each transaction
df['Zero Digit Ratio Amount Received'] = df['Amount Received'].apply(zero_digit_ratio)
df['Zero Digit Ratio Amount Paid'] = df['Amount Paid'].apply(zero_digit_ratio)

# Calculate the average zero digit ratio for paid transactions by each account
paid_zero_digit_ratio_stats = df.groupby('From Account')['Zero Digit Ratio Amount Paid'].mean().reset_index()
paid_zero_digit_ratio_stats.rename(columns={
    'Zero Digit Ratio Amount Paid': 'Average Zero Digit Ratio Paid'
}, inplace=True)

# Calculate the average zero digit ratio for received transactions by each account
received_zero_digit_ratio_stats = df.groupby('To Account')['Zero Digit Ratio Amount Received'].mean().reset_index()
received_zero_digit_ratio_stats.rename(columns={
    'Zero Digit Ratio Amount Received': 'Average Zero Digit Ratio Received'
}, inplace=True)

# Merge both data sets into a single data frame
zero_digit_ratio_stats = pd.merge(paid_zero_digit_ratio_stats, received_zero_digit_ratio_stats, how='outer', left_on='From Account', right_on='To Account')

# Combine the two account IDs in a new variable "Account ID"
zero_digit_ratio_stats['Account ID'] = zero_digit_ratio_stats['From Account'].combine_first(zero_digit_ratio_stats['To Account'])

# Drop redundant columns
zero_digit_ratio_stats = zero_digit_ratio_stats.drop(columns=['From Account', 'To Account'])
zero_digit_ratio_stats.head()

,Average Zero Digit Ratio Paid,Average Zero Digit Ratio Received,Account ID
0,0.470215,0.498400,100428660
1,0.475983,0.498757,1004286A8
2,0.443252,0.458391,1004286F0
3,0.407221,0.439213,100428738
4,0.408176,0.435559,100428780


In [16]:
# Merge with variable "From Account"
df_v2 = pd.merge(df_v1, zero_digit_ratio_stats, how='left', left_on='From Account', right_on='Account ID')
df_v2.rename(columns={
    'Average Zero Digit Ratio Paid': 'From Account Average Zero Digit Ratio Paid',
    'Average Zero Digit Ratio Received': 'From Account Average Zero Digit Ratio Received'
}, inplace=True)
df_v2.drop(columns=['Account ID'], inplace=True)

# Merge with variable "To Account"
df_v2 = pd.merge(df_v2, zero_digit_ratio_stats, how='left', left_on='To Account', right_on='Account ID')
df_v2.rename(columns={
    'Average Zero Digit Ratio Paid': 'To Account Average Zero Digit Ratio Paid',
    'Average Zero Digit Ratio Received': 'To Account Average Zero Digit Ratio Received'
}, inplace=True)
df_v2.drop(columns=['Account ID'], inplace=True)
df_v2.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,47.64,Saudi Riyal,47.64,Saudi Riyal,Reinvestment,...,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,3917.42,Canadian Dollar,3917.42,Canadian Dollar,Reinvestment,...,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,97.49,Canadian Dollar,97.49,Canadian Dollar,Credit Card,...,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153
3,2022-09-01,32317,800D4E490,12004,800D4E750,13939.05,Euro,13939.05,Euro,Wire,...,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.37,Euro,10.37,Euro,Reinvestment,...,19.250000,4.500000,19.333333,4.766667,19.250000,4.500000,0.595648,0.517361,0.595648,0.517361


In [17]:
# CHECK RESULTS - Average ratio of zeros compared to total number of digits by a selected account

# Check whether ratio of zeros are correct calculated for a specific account
df_v2_test = df[df['From Account'] == '100428780']
df_v2_test.head(20) # to control if a single transaction is correctly calculated
df_v2_test.describe() # to find average

# Check with the corresponding variables
df_v2_test2 = df_v2[df_v2['From Account'] == '100428780']
df_v2_test2.head()


,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received
8830,2022-09-01 00:00:00,70,100428780,113304,805C41320,2304.14,Rupee,2304.14,Rupee,Credit Card,...,29.290598,4.632479,16.000000,4.000000,24.866667,4.466667,0.408176,0.435559,0.444444,0.432660
9125,2022-09-01 00:00:00,70,100428780,113530,8056886E0,740240.35,Rupee,740240.35,Rupee,Cheque,...,29.290598,4.632479,30.566667,4.733333,31.333333,4.185185,0.408176,0.435559,0.430455,0.396886
9577,2022-09-01 00:00:00,70,100428780,212789,8050C39D0,9212380.32,Rupee,9212380.32,Rupee,Cash,...,29.290598,4.632479,39.000000,4.000000,33.933333,4.800000,0.408176,0.435559,0.376068,0.377225
9677,2022-09-01 00:00:00,70,100428780,14629,8059211D0,665742.62,Rupee,665742.62,Rupee,Credit Card,...,29.290598,4.632479,21.000000,4.000000,37.821429,4.785714,0.408176,0.435559,0.400000,0.404426
12158,2022-09-01 00:01:00,70,100428780,112733,805C2A320,3095.91,Rupee,3095.91,Rupee,Cheque,...,29.290598,4.632479,38.000000,5.000000,21.333333,5.476190,0.408176,0.435559,0.416667,0.517677


### Convert all Amounts into US-Dollars (for comparability)

In [18]:
# Convert all currencies to US-Dollars (exchange rates from 09/2022, waehrungsrechner.org)
currency_multipliers = {
    'Australian Dollar': 0.67926,
    'Bitcoin': 20000,
    'Brazil Real': 0.19256,
    'Canadian Dollar': 0.76011,
    'Euro': 0.99622,
    'Mexican Peso': 0.05031,
    'Ruble': 0.01673,
    'Rupee': 0.01255,
    'Saudi Riyal': 0.26612,
    'Shekel': 0.29351,
    'Swiss Franc': 0.98166,
    'UK Pound': 1.15331,
    'Yen': 0.00713,
    'Yuan': 0.14446
}
for currency, multiplier in currency_multipliers.items():
    df_v2.loc[df_v2['Payment Currency'] == currency, 'Amount Paid'] *= multiplier
    df_v2.loc[df_v2['Receiving Currency'] == currency, 'Amount Received'] *= multiplier
    
df_v2.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,...,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,...,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,...,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,...,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.330801,Euro,10.330801,Euro,Reinvestment,...,19.250000,4.500000,19.333333,4.766667,19.250000,4.500000,0.595648,0.517361,0.595648,0.517361


In [19]:
# Calculate descriptive statistics of "Amount Paid"
df_v2['Amount Paid'].describe(), df_v2.groupby('Is Laundering')['Amount Paid'].describe()

(count    5.078345e+06
 mean     3.436240e+05
 std      2.456332e+07
 min      7.130000e-05
 25%      1.521100e+02
 50%      8.616000e+02
 75%      5.147710e+03
 max      2.887692e+10
 Name: Amount Paid, dtype: float64,
                    count          mean           std       min      25%  \
 Is Laundering                                                             
 0              5073168.0  3.387590e+05  2.343559e+07  0.000071   151.90   
 1                 5177.0  5.111051e+06  2.315927e+08  0.328753  2041.64   
 
                        50%           75%           max  
 Is Laundering                                           
 0               859.239750   5133.103616  2.887692e+10  
 1              5668.880326  12858.473980  1.634289e+10  )

### V3: Dynamic Variables: Min, Mean, Median, Max, Sum & Count

In [20]:
# Create dynamic Min, Mean, Median, Max, Sum & Count for every sender account ("From Account")
df_v3 = df_v2.copy()
df_v3['From Account Amount Paid Min'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().min())
df_v3['From Account Amount Paid Mean'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().mean())
df_v3['From Account Amount Paid Median'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().median())
df_v3['From Account Amount Paid Max'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().max())
df_v3['From Account Amount Paid Sum'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().sum())
df_v3['From Account Amount Paid Count'] = df_v3.groupby('From Account')['Amount Paid'].transform(lambda x: x.expanding().count())

# Create dynamic Min, Mean, Median, Max, Sum & Count for every receiver account ("To Account")
df_v3['To Account Amount Received Min'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().min())
df_v3['To Account Amount Received Mean'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().mean())
df_v3['To Account Amount Received Median'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().median())
df_v3['To Account Amount Received Max'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().max())
df_v3['To Account Amount Received Sum'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().sum())
df_v3['To Account Amount Received Count'] = df_v3.groupby('To Account')['Amount Received'].transform(lambda x: x.expanding().count())

df_v3.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,...,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,...,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,...,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,...,13886.360391,13886.360391,13886.360391,1.0,13886.360391,13886.360391,13886.360391,13886.360391,13886.360391,1.0
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.330801,Euro,10.330801,Euro,Reinvestment,...,10.330801,10.330801,10.330801,1.0,10.330801,10.330801,10.330801,10.330801,10.330801,1.0


In [21]:
# CHECK RESULTS - Dynmamic variables by a selected account history
pd.set_option('display.max_columns', None)
df_v3_test_from = df_v3[df_v3['From Account'] == '100428780']
df_v3_test_from.head()

df_v3_test_to = df_v3[df_v3['To Account'] == '100428780']
df_v3_test_to.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count
114023,2022-09-01 00:10:00,313638,804F4AED0,70,100428780,0.001380,Rupee,0.001380,Rupee,Cash,0,2,2,5,5,16.666667,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.601732,NaN,0.408176,0.435559,0.001380,0.001380,0.001380,0.001380,0.001380,1.0,0.001380,0.001380,0.001380,0.001380,0.001380,1.0
124189,2022-09-01 00:11:00,312747,8054CB880,70,100428780,0.001255,Rupee,0.001255,Rupee,Cash,0,1,1,6,6,19.000000,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.528139,NaN,0.408176,0.435559,0.001255,0.001255,0.001255,0.001255,0.001255,1.0,0.001255,0.001318,0.001318,0.001380,0.002635,2.0
139045,2022-09-01 00:12:00,312246,805410180,70,100428780,0.000376,Rupee,0.000376,Rupee,Cash,0,3,3,6,6,23.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.552381,NaN,0.408176,0.435559,0.000376,0.000376,0.000376,0.000376,0.000376,1.0,0.000376,0.001004,0.001255,0.001380,0.003012,3.0
465982,2022-09-01 04:38:00,312830,80536CA40,70,100428780,0.001004,Rupee,0.001004,Rupee,Cash,0,8,8,6,6,22.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.528139,NaN,0.408176,0.435559,0.001004,0.001004,0.001004,0.001004,0.001004,1.0,0.000376,0.001004,0.001130,0.001380,0.004016,4.0
724918,2022-09-01 12:22:00,313167,804CDF110,70,100428780,0.005899,Rupee,0.005899,Rupee,Cash,0,11,11,5,5,14.333333,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.571429,NaN,0.408176,0.435559,0.005899,0.005899,0.005899,0.005899,0.005899,1.0,0.000376,0.001983,0.001255,0.005899,0.009914,5.0


### V4: Average Sum per Payment Format

In [22]:
# Calculate average amount paid per payment format grouped by "From Account"
average_paid = df_v3.pivot_table(values='Amount Paid', index='From Account', columns='Payment Format', aggfunc='mean')
average_paid.columns = [f'From Account Average Amount Paid in {col}' for col in average_paid.columns]

# Calculate average amount received per payment format grouped by "To Account"
average_received = df_v3.pivot_table(values='Amount Received', index='To Account', columns='Payment Format', aggfunc='mean')
average_received.columns = [f'To Account Average Amount Received in {col}' for col in average_received.columns]

# Merge with variables "From Account" and "To Account"
df_v4 = pd.merge(df_v3, average_paid, how='left', left_on='From Account', right_index=True)
df_v4 = pd.merge(df_v4, average_received, how='left', left_on='To Account', right_index=True)
df_v4.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,0,21,21,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,2058.070954,NaN,NaN,NaN,NaN,113.920650,NaN,NaN,NaN,NaN,2.543906e+03,27771.013808,113.920650,NaN
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,0,26,26,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2089.542390,NaN,NaN,NaN,NaN,1493.562942,NaN,NaN,NaN,NaN,NaN,NaN,1493.562942,NaN
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,0,29,29,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,NaN,NaN,NaN,NaN,63.796792,NaN,NaN,156.286217,NaN,3659.92965,5.899356e+04,52.150456,34307.393825,NaN
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,0,30,30,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417,13886.360391,13886.360391,13886.360391,13886.360391,13886.360391,1.0,13886.360391,13886.360391,13886.360391,13886.360391,13886.360391,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13886.360391,NaN,NaN,NaN,1.370731e+06,NaN,2514.434374,13886.360391
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.330801,Euro,10.330801,Euro,Reinvestment,0,11,11,5,5,19.333333,4.766667,19.250000,4.500000,19.333333,4.766667,19.250000,4.500000,0.595648,0.517361,0.595648,0.517361,10.330801,10.330801,10.330801,10.330801,10.330801,1.0,10.330801,10.330801,10.330801,10.330801,10.330801,1.0,NaN,NaN,1876.450105,273.557031,0.368601,264.361920,NaN,NaN,NaN,NaN,1.767892e+02,NaN,264.361920,NaN


In [23]:
# CHECK RESULTS - Average sum per payment format by a selected account
df_v4_test_from = df_v4[df_v4['From Account'] == '100428780']
df_v4_test_from_grouped = df_v4_test_from.groupby('Payment Format')['Amount Paid'].mean()
df_v4_test_from_grouped
df_v4_test_from.head()

df_v4_test_to = df_v4[df_v4['To Account'] == '100428780']
df_v4_test_to_grouped = df_v4_test_to.groupby('Payment Format')['Amount Received'].mean()
df_v4_test_to_grouped
df_v4_test_to.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire
114023,2022-09-01 00:10:00,313638,804F4AED0,70,100428780,0.001380,Rupee,0.001380,Rupee,Cash,0,2,2,5,5,16.666667,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.601732,NaN,0.408176,0.435559,0.001380,0.001380,0.001380,0.001380,0.001380,1.0,0.001380,0.001380,0.001380,0.001380,0.001380,1.0,NaN,NaN,0.001380,NaN,201.295725,NaN,NaN,NaN,NaN,1133.886059,NaN,NaN,NaN,NaN
124189,2022-09-01 00:11:00,312747,8054CB880,70,100428780,0.001255,Rupee,0.001255,Rupee,Cash,0,1,1,6,6,19.000000,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.528139,NaN,0.408176,0.435559,0.001255,0.001255,0.001255,0.001255,0.001255,1.0,0.001255,0.001318,0.001318,0.001380,0.002635,2.0,146.786557,NaN,0.001255,NaN,NaN,NaN,NaN,NaN,NaN,1133.886059,NaN,NaN,NaN,NaN
139045,2022-09-01 00:12:00,312246,805410180,70,100428780,0.000376,Rupee,0.000376,Rupee,Cash,0,3,3,6,6,23.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.552381,NaN,0.408176,0.435559,0.000376,0.000376,0.000376,0.000376,0.000376,1.0,0.000376,0.001004,0.001255,0.001380,0.003012,3.0,87.045796,NaN,0.000376,NaN,NaN,NaN,NaN,NaN,NaN,1133.886059,NaN,NaN,NaN,NaN
465982,2022-09-01 04:38:00,312830,80536CA40,70,100428780,0.001004,Rupee,0.001004,Rupee,Cash,0,8,8,6,6,22.666667,4.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.528139,NaN,0.408176,0.435559,0.001004,0.001004,0.001004,0.001004,0.001004,1.0,0.000376,0.001004,0.001130,0.001380,0.004016,4.0,NaN,NaN,0.001004,767.903376,NaN,NaN,NaN,NaN,NaN,1133.886059,NaN,NaN,NaN,NaN
724918,2022-09-01 12:22:00,313167,804CDF110,70,100428780,0.005899,Rupee,0.005899,Rupee,Cash,0,11,11,5,5,14.333333,5.666667,NaN,NaN,32.097949,4.635832,29.290598,4.632479,0.571429,NaN,0.408176,0.435559,0.005899,0.005899,0.005899,0.005899,0.005899,1.0,0.000376,0.001983,0.001255,0.005899,0.009914,5.0,NaN,NaN,0.005899,1644.310036,NaN,NaN,NaN,NaN,NaN,1133.886059,NaN,NaN,NaN,NaN


### V5: Transaction Count

In [24]:
# Count number of transactions each account received
received_counts = df_v2.groupby('To Account').size().reset_index(name='Received Transaction Count')

# Count number of transactions each account paid
paid_counts = df_v2.groupby('From Account').size().reset_index(name='Paid Transaction Count')

# Merge both datasets into a single data frame
transaction_counts = pd.merge(paid_counts, received_counts, how='outer', left_on='From Account', right_on='To Account')

# Combine the two account IDs in a new variable "Account ID"
transaction_counts['Account ID'] = transaction_counts['From Account'].combine_first(transaction_counts['To Account'])

# Drop redundant columns
transaction_counts = transaction_counts.drop(columns=['From Account', 'To Account'])
transaction_counts.head()

,Paid Transaction Count,Received Transaction Count,Account ID
0,168672.0,1084.0,100428660
1,103018.0,653.0,1004286A8
2,18663.0,108.0,1004286F0
3,13756.0,98.0,100428738
4,17264.0,117.0,100428780


In [25]:
# Merge with variable "From Account"
df_v5 = pd.merge(df_v4, transaction_counts, how='left', left_on='From Account', right_on='Account ID')
df_v5.rename(columns={
    'Paid Transaction Count': 'From Account Paid Transaction Count',
    'Received Transaction Count': 'From Account Received Transaction Count'
}, inplace=True)
df_v5.drop(columns=['Account ID'], inplace=True)

# Merge with variable "To Account"
df_v5 = pd.merge(df_v5, transaction_counts, how='left', left_on='To Account', right_on='Account ID')
df_v5.rename(columns={
    'Paid Transaction Count': 'To Account Paid Transaction Count',
    'Received Transaction Count': 'To Account Received Transaction Count'
}, inplace=True)
df_v5.drop(columns=['Account ID'], inplace=True)
df_v5.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,0,21,21,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,2058.070954,NaN,NaN,NaN,NaN,113.920650,NaN,NaN,NaN,NaN,2.543906e+03,27771.013808,113.920650,NaN,3.0,14.0,3.0,14.0
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,0,26,26,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2089.542390,NaN,NaN,NaN,NaN,1493.562942,NaN,NaN,NaN,NaN,NaN,NaN,1493.562942,NaN,3.0,2.0,3.0,2.0
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,0,29,29,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,NaN,NaN,NaN,NaN,63.796792,NaN,NaN,156.286217,NaN,3659.92965,5.899356e+04,52.150456,34307.393825,NaN,10.0,1.0,57.0,41.0
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,0,30,30,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417,13886.360391,13886.360391,13886.360391,13886.360391,13886.360391,1.0,13886.360391,13886.360391,13886.360391,13886.360391,13886.360391,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13886.360391,NaN,NaN,NaN,1.370731e+06,NaN,2514.434374,13886.360391,1.0,NaN,9.0,4.0
4,2022-09-01,1024,800C8D9D0,1024,800C8D9D0,10.330801,Euro,10.330801,Euro,Reinvestment,0,11,11,5,5,19.333333,4.766667,19.250000,4.500000,19.333333,4.766667,19.250000,4.500000,0.595648,0.517361,0.595648,0.517361,10.330801,10.330801,10.330801,10.330801,10.330801,1.0,10.330801,10.330801,10.330801,10.330801,10.330801,1.0,NaN,N

In [26]:
# CHECK RESULTS - Transaction counts by a selected account
df_v5_test_from = df_v5[df_v5['From Account'] == '100428780']
df_v5_test_from.describe()

df_v5_test_to = df_v5[df_v5['To Account'] == '100428780']
df_v5_test_to.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count
count,117.000000,117.0,117.000000,117.000000,117.0,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,2.00,2.00,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02,117.000000,2.000000,1.170000e+02,1.170000e+02,117.000000,117.000000,117.000000,117.000000,1.170000e+02,117.00000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,14.000000,0.0,117.000000,52.000000,3.000000,2.000000,2.000000,0.0,0.0,1.170000e+02,0.0,0.0,0.0,0.0,117.000000,2.0,117.0,117.0
mean,308834.717949,70.0,1133.886059,1133.886059,0.0,29.290598,29.290598,4.632479,4.632479,25.791311,4.770522,28.48,5.32,3.209795e+01,4.635832e+00,2.929060e+01,4.632479e+00,0.490891,0.447532,4.081758e-01,4.355594e-01,129.080259,815.917475,423.021237,8694.987857,3.079626e+04,3.25641,0.000393,752.697378,212.283333,26526.107806,53928.224591,59.000000,7576.695353,NaN,685.540440,423.584535,1725.674976,245802.832905,145577.290832,NaN,NaN,1.133886e+03,NaN,NaN,NaN,NaN,4.709402,25.0,17264.0,117.0
std,41180.572256,0.0,5194.969419,5194.969419,0.0,9.381805,9.381805,0.749693,0.749693,8.504403,0.628768,0.00,0.00,9.990384e-14,4.459993e-15,2.854395e-14,3.567994e-15,0.086586,0.000000,8.362486e-16,9.477485e-16,188.562429,4051.570828,658.701679,62734.303088,2.339761e+05,10.97030,0.000123,390.365535,78.188492,18444.188785,39049.911301,33.919021,19210.139283,NaN,1891.576814,1911.365473,1320.151156,0.000000,0.000000,NaN,NaN,1.370110e-12,NaN,NaN,NaN,NaN,16.336473,0.0,0.0,0.0
min,16.000000,70.0,0.000376,0.000376,0.0,1.000000,1.000000,4.000000,4.000000,5.666667,4.000000,28.48,5.32,3.209795e+01,4.635832e+00,2.929060e+01,4.632479e+00,0.363636,0.447532,4.081758e-01,4.355594e-01,0.000126,0.000376,0.000376,0.000376,3.765000e-04,1.00000,0.000376,0.001004,0.001130,0.001380,0.001380,1.000000,0.000126,NaN,0.000376,0.000126,201.295725,245802.832905,145577.290832,NaN,NaN,1.133886e+03,NaN,NaN,NaN,NaN,2.000000,25.0,17264.0,117.0
25%,312747.000000,70.0,146.534302,146.534302,0.0,25.000000,25.000000,4.000000,4.000000,20.000000,4.333333,28.48,5.32,3.209795e+01,4.635832e+00,2.929060e+01,4.632479e+00,0.444444,0.447532,4.081758e-01,4.355594e-01,0.000628,100.057699,122.537886,146.534302,1.972056e+02,1.00000,0.000376,351.821534,200.115021,2026.991

### V6: Standard Deviations & Average Amounts

In [27]:
# Calculate the average and standard deviation of the amount paid by each account
paid_stats = df_v2.groupby('From Account')['Amount Paid'].agg(['mean', 'std']).reset_index()
paid_stats.rename(columns={
    'mean': 'Average Amount Paid',
    'std': 'Std Dev Amount Paid'
}, inplace=True)

# Calculate the average and standard deviation of the amount received by each account
received_stats = df_v2.groupby('To Account')['Amount Received'].agg(['mean', 'std']).reset_index()
received_stats.rename(columns={
    'mean': 'Average Amount Received',
    'std': 'Std Dev Amount Received'
}, inplace=True)

# Merge both data sets into a single data frame
account_stats = pd.merge(paid_stats, received_stats, how='outer', left_on='From Account', right_on='To Account')

# Combine the two account IDs in a new variable "Account ID"
account_stats['Account ID'] = account_stats['From Account'].combine_first(account_stats['To Account'])

# Drop redundant columns
account_stats = account_stats.drop(columns=['From Account', 'To Account'])
account_stats.head()

,Average Amount Paid,Std Dev Amount Paid,Average Amount Received,Std Dev Amount Received,Account ID
0,312810.017132,8.093752e+06,442.430618,637.651237,100428660
1,252099.955717,5.855185e+06,382.928768,580.761371,1004286A8
2,190859.017711,2.860530e+06,339.660077,417.590047,1004286F0
3,134357.055367,2.984029e+06,458.676425,679.765108,100428738
4,271829.699056,5.405056e+06,1133.886059,5194.969419,100428780


In [28]:
# Merge with variable "From Account"
df_v6 = pd.merge(df_v5, account_stats, how='left', left_on='From Account', right_on='Account ID')
df_v6.rename(columns={
    'Average Amount Paid': 'From Account Average Amount Paid',
    'Std Dev Amount Paid': 'From Account Std Dev Amount Paid',
    'Average Amount Received': 'From Account Average Amount Received',
    'Std Dev Amount Received': 'From Account Std Dev Amount Received'
}, inplace=True)
df_v6.drop(columns=['Account ID'], inplace=True)

# Merge with variable "To Account"
df_v6 = pd.merge(df_v6, account_stats, how='left', left_on='To Account', right_on='Account ID')
df_v6.rename(columns={
    'Average Amount Paid': 'To Account Average Amount Paid',
    'Std Dev Amount Paid': 'To Account Std Dev Amount Paid',
    'Average Amount Received': 'To Account Average Amount Received',
    'Std Dev Amount Received': 'To Account Std Dev Amount Received'
}, inplace=True)
df_v6.drop(columns=['Account ID'], inplace=True)
df_v6.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count,From Account Average Amount Paid,From Account Std Dev Amount Paid,From Account Average Amount Received,From Account Std Dev Amount Received,To Account Average Amount Paid,To Account Std Dev Amount Paid,To Account Average Amount Received,To Account Std Dev Amount Received
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,0,21,21,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,2058.070954,NaN,NaN,NaN,NaN,113.920650,NaN,NaN,NaN,NaN,2.543906e+03,27771.013808,113.920650,NaN,3.0,14.0,3.0,14.0,761.970751,1127.012371,3998.701632,7731.948289,761.970751,1127.012371,3998.701632,7731.948289
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,0,26,26,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2089.542390,NaN,NaN,NaN,NaN,1493.562942,NaN,NaN,NaN,NaN,NaN,NaN,1493.562942,NaN,3.0,2.0,3.0,2.0,1692.222758,1523.473423,1493.562942,2098.844493,1692.222758,1523.473423,1493.562942,2098.844493
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,0,29,29,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,NaN,NaN,NaN,NaN,63.796792,NaN,NaN,156.286217,NaN,3659.92965,5.899356e+04,52.150456,34307.393825,NaN,10.0,1.0,57.0,41.0,63.796792,85.324934,2089.542390,NaN,6466.005467,11491.606058,13844.942578,69680.093067
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,0,30,30,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417,13886.360391,13886.

In [29]:
# CHECK RESULTS - Standard deviations and average amounts by a selected account
df_v6_test_from = df_v6[df_v6['From Account'] == '100428780']
df_v6_test_from.describe()

df_v6_test_to = df_v6[df_v6['To Account'] == '100428780']
df_v6_test_to.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count,From Account Average Amount Paid,From Account Std Dev Amount Paid,From Account Average Amount Received,From Account Std Dev Amount Received,To Account Average Amount Paid,To Account Std Dev Amount Paid,To Account Average Amount Received,To Account Std Dev Amount Received
count,117.000000,117.0,117.000000,117.000000,117.0,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,2.00,2.00,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02,117.000000,2.000000,1.170000e+02,1.170000e+02,117.000000,117.000000,117.000000,117.000000,1.170000e+02,117.00000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,14.000000,0.0,117.000000,52.000000,3.000000,2.000000,2.000000,0.0,0.0,1.170000e+02,0.0,0.0,0.0,0.0,117.000000,2.0,117.0,117.0,117.000000,117.000000,2.000000,2.000000,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02
mean,308834.717949,70.0,1133.886059,1133.886059,0.0,29.290598,29.290598,4.632479,4.632479,25.791311,4.770522,28.48,5.32,3.209795e+01,4.635832e+00,2.929060e+01,4.632479e+00,0.490891,0.447532,4.081758e-01,4.355594e-01,129.080259,815.917475,423.021237,8694.987857,3.079626e+04,3.25641,0.000393,752.697378,212.283333,26526.107806,53928.224591,59.000000,7576.695353,NaN,685.540440,423.584535,1725.674976,245802.832905,145577.290832,NaN,NaN,1.133886e+03,NaN,NaN,NaN,NaN,4.709402,25.0,17264.0,117.0,645.110133,1265.040502,45776.805622,97000.360817,2.718297e+05,5.405056e+06,1.133886e+03,5.194969e+03
std,41180.572256,0.0,5194.969419,5194.969419,0.0,9.381805,9.381805,0.749693,0.749693,8.504403,0.628768,0.00,0.00,9.990384e-14,4.459993e-15,2.854395e-14,3.567994e-15,0.086586,0.000000,8.362486e-16,9.477485e-16,188.562429,4051.570828,658.701679,62734.303088,2.339761e+05,10.97030,0.000123,390.365535,78.188492,18444.188785,39049.911301,33.919021,19210.139283,NaN,1891.576814,1911.365473,1320.151156,0.000000,0.000000,NaN,NaN,1.370110e-12,NaN,NaN,NaN,NaN,16.336473,0.0,0.0,0.0,2179.406745,8100.420297,0.000000,0.000000,5.845802e-10,1.215927e-08,1.370110e-12,1.278769e-11
min,16.000000,70.0,0.000376,0.000376,0.0,1.000000,1.000000,4.000000,4.000000,5.666667,4.000000,28.48,5.32,3.209795e+01,4.635832e+00,2.929060e+01,4.632479e+00,0.363636,0.447532,4.081758e-01,4.355594e-01,0.

### V7: Indication of Transaction was within the Bank

In [30]:
# New dummy variable "Within Bank" indicating if transaction occured within bank or not
df_v7 = df_v6.copy()
df_v7['Within Bank'] = (df_v7['From Bank'].astype(str) == df_v7['To Bank'].astype(str)).astype(int)
df_v7.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count,From Account Average Amount Paid,From Account Std Dev Amount Paid,From Account Average Amount Received,From Account Std Dev Amount Received,To Account Average Amount Paid,To Account Std Dev Amount Paid,To Account Average Amount Received,To Account Std Dev Amount Received,Within Bank
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,0,21,21,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,2058.070954,NaN,NaN,NaN,NaN,113.920650,NaN,NaN,NaN,NaN,2.543906e+03,27771.013808,113.920650,NaN,3.0,14.0,3.0,14.0,761.970751,1127.012371,3998.701632,7731.948289,761.970751,1127.012371,3998.701632,7731.948289,1
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,0,26,26,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2089.542390,NaN,NaN,NaN,NaN,1493.562942,NaN,NaN,NaN,NaN,NaN,NaN,1493.562942,NaN,3.0,2.0,3.0,2.0,1692.222758,1523.473423,1493.562942,2098.844493,1692.222758,1523.473423,1493.562942,2098.844493,1
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,0,29,29,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,NaN,NaN,NaN,NaN,63.796792,NaN,NaN,156.286217,NaN,3659.92965,5.899356e+04,52.150456,34307.393825,NaN,10.0,1.0,57.0,41.0,63.796792,85.324934,2089.542390,NaN,6466.005467,11491.606058,13844.942578,69680.093067,0
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,0,30,30,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.500000,0.454545,NaN,0.485602,0.458417,1

### V8: Extraction of "day", "hour" and "minute" from "Timestamp"

In [31]:
# Extract time components from variable "Timestamp"
df_v8 = df_v7.copy()
df_v8['Day'] = df_v8['Timestamp'].dt.day
df_v8['Hour'] = df_v8['Timestamp'].dt.hour
df_v8['Minute'] = df_v8['Timestamp'].dt.minute
df_v8.head()

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Digit Sum Amount Received,Digit Sum Amount Paid,Count Zeros Amount Received,Count Zeros Amount Paid,From Account Average Digit Sum Paid,From Account Average Count Zeros Paid,From Account Average Digit Sum Received,From Account Average Count Zeros Received,To Account Average Digit Sum Paid,To Account Average Count Zeros Paid,To Account Average Digit Sum Received,To Account Average Count Zeros Received,From Account Average Zero Digit Ratio Paid,From Account Average Zero Digit Ratio Received,To Account Average Zero Digit Ratio Paid,To Account Average Zero Digit Ratio Received,From Account Amount Paid Min,From Account Amount Paid Mean,From Account Amount Paid Median,From Account Amount Paid Max,From Account Amount Paid Sum,From Account Amount Paid Count,To Account Amount Received Min,To Account Amount Received Mean,To Account Amount Received Median,To Account Amount Received Max,To Account Amount Received Sum,To Account Amount Received Count,From Account Average Amount Paid in ACH,From Account Average Amount Paid in Bitcoin,From Account Average Amount Paid in Cash,From Account Average Amount Paid in Cheque,From Account Average Amount Paid in Credit Card,From Account Average Amount Paid in Reinvestment,From Account Average Amount Paid in Wire,To Account Average Amount Received in ACH,To Account Average Amount Received in Bitcoin,To Account Average Amount Received in Cash,To Account Average Amount Received in Cheque,To Account Average Amount Received in Credit Card,To Account Average Amount Received in Reinvestment,To Account Average Amount Received in Wire,From Account Paid Transaction Count,From Account Received Transaction Count,To Account Paid Transaction Count,To Account Received Transaction Count,From Account Average Amount Paid,From Account Std Dev Amount Paid,From Account Average Amount Received,From Account Std Dev Amount Received,To Account Average Amount Paid,To Account Std Dev Amount Paid,To Account Average Amount Received,To Account Std Dev Amount Received,Within Bank,Day,Hour,Minute
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,12.677957,Saudi Riyal,12.677957,Saudi Riyal,Reinvestment,0,21,21,4,4,24.000000,4.333333,26.071429,4.857143,24.000000,4.333333,26.071429,4.857143,0.485185,0.488276,0.485185,0.488276,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,12.677957,12.677957,12.677957,12.677957,12.677957,1.0,2058.070954,NaN,NaN,NaN,NaN,113.920650,NaN,NaN,NaN,NaN,2.543906e+03,27771.013808,113.920650,NaN,3.0,14.0,3.0,14.0,761.970751,1127.012371,3998.701632,7731.948289,761.970751,1127.012371,3998.701632,7731.948289,1,1,0,0
1,2022-09-01,25170,8095AF7C0,25170,8095AF7C0,2977.670116,Canadian Dollar,2977.670116,Canadian Dollar,Reinvestment,0,26,26,4,4,19.666667,4.666667,18.500000,4.000000,19.666667,4.666667,18.500000,4.000000,0.500000,0.450000,0.500000,0.450000,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2977.670116,2977.670116,2977.670116,2977.670116,2977.670116,1.0,2089.542390,NaN,NaN,NaN,NaN,1493.562942,NaN,NaN,NaN,NaN,NaN,NaN,1493.562942,NaN,3.0,2.0,3.0,2.0,1692.222758,1523.473423,1493.562942,2098.844493,1692.222758,1523.473423,1493.562942,2098.844493,1,1,0,0
2,2022-09-01,25665,809A7D4B0,24779,809189BA0,74.103124,Canadian Dollar,74.103124,Canadian Dollar,Credit Card,0,29,29,4,4,17.700000,4.700000,22.000000,6.000000,27.017544,4.438596,18.463415,4.658537,0.582143,0.600000,0.450642,0.527153,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,74.103124,74.103124,74.103124,74.103124,74.103124,1.0,NaN,NaN,NaN,NaN,63.796792,NaN,NaN,156.286217,NaN,3659.92965,5.899356e+04,52.150456,34307.393825,NaN,10.0,1.0,57.0,41.0,63.796792,85.324934,2089.542390,NaN,6466.005467,11491.606058,13844.942578,69680.093067,0,1,0,0
3,2022-09-01,32317,800D4E490,12004,800D4E750,13886.360391,Euro,13886.360391,Euro,Wire,0,30,30,5,5,30.000000,5.000000,NaN,NaN,26.000000,4.888889,25.500000,4.50000

### Save File with all Variables

In [32]:
df_v8.to_csv('./df_all_var.csv', index=False)